In [1]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
%matplotlib inline 
warnings.filterwarnings("ignore")

from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
from keras.models import Sequential



Using Theano backend.
C:\Users\sran12\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
def get_im_cv2(path):
    img = cv2.imread(path)
    height, width, channels = img.shape
    img = cv2.resize(img, (64, 64))
    return img

In [28]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    yfl = []
    start_time = time.time()

    print('Read train images')
    folders = ['Automobile',
               'Comics_and_Cartoons',
               'Famous_Personalities',
               'Festivals_and_Occasions',
               'Graffiti_and_Illustrations',
               'Movies_and_TV_shows',
               'Music',
               'Nature',
               'No_Theme',
               'Patterns_and_Ethnic',
               'Signs_and_Symbols',
               'Spiritual',
               'Sports',
               'Superheroes',
               'Typography',
               'Vintage'
              ]
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Train', fld, '*.jpg')
        files = glob.glob(path)
        for i in range(len(files)):
            flbase = os.path.basename(files[i])
            img = get_im_cv2(files[i])
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)
            yfl= files[i].split("\\")[6]
    print len(X_train)
    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id,yfl

In [29]:
def load_test():
    path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [30]:
Xtrall, Ytrall, Itrall,yfl = load_train()
X_test, X_test_id = load_test()

Read train images
Load folder Automobile (Index: 0)
Load folder Comics_and_Cartoons (Index: 1)
Load folder Famous_Personalities (Index: 2)
Load folder Festivals_and_Occasions (Index: 3)
Load folder Graffiti_and_Illustrations (Index: 4)
Load folder Movies_and_TV_shows (Index: 5)
Load folder Music (Index: 6)
Load folder Nature (Index: 7)
Load folder No_Theme (Index: 8)
Load folder Patterns_and_Ethnic (Index: 9)
Load folder Signs_and_Symbols (Index: 10)
Load folder Spiritual (Index: 11)
Load folder Sports (Index: 12)
Load folder Superheroes (Index: 13)
Load folder Typography (Index: 14)
Load folder Vintage (Index: 15)
4591
Read train data time: 171.48 seconds


In [33]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(Ytrall), Ytrall)

In [35]:
# flatten out all images to be one-dimensional
Xtrall = np.array(Xtrall, dtype=np.uint8)
Ytrall = np.array(Ytrall, dtype=np.uint8)

Xtrall = Xtrall.astype('float32')
Xtrall = Xtrall / 255
Xtrall = Xtrall.transpose((0, 3, 1, 2))

X_test = np.array(X_test, dtype=np.uint8)

X_test = X_test.astype('float32')
X_test = X_test / 255
X_test = X_test.transpose((0, 3, 1, 2))

Ytrall = np_utils.to_categorical(Ytrall, 16)

In [41]:
batch_size=64
nb_classes=len(Ytrall)
nb_epoch=40
nb_filters=64
nb_pool=1
nb_conv=3

In [42]:
model= Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=Xtrall.shape[1:]))
model.add(Convolution2D(nb_filters,nb_conv,nb_conv, subsample=(1, 1),activation='relu'))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters,nb_conv,nb_conv, subsample=(1, 1),activation='relu'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)))
model.add(Dropout(0.3));
model.add(Flatten());
model.add(Dense(64));
model.add(Dropout(0.5));
model.add(Dense(16));
model.add(Activation('softmax'));
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])


In [43]:
nfolds = 20
yfull_train = dict()
kf = KFold(len(Itrall), n_folds=nfolds, shuffle=True, random_state=51)
num_fold = 0
sum_score = 0
models = []

for mtr,mval in kf:
    Xtr = Xtrall[mtr]
    Ytr = Ytrall[mtr]
    Xval = Xtrall[mval]
    Yval = Ytrall[mval]
    
    num_fold += 1
    
    print('Start KFold number {} from {}'.format(num_fold, nfolds))
    print('Split train: ', len(Xtr), len(Ytr))
    print('Split valid: ', len(Xval), len(Yval))
    
    callbacks = [
            EarlyStopping(monitor='val_loss', patience=4, verbose=0),
        ]
    model.fit(Xtr,Ytr, batch_size=batch_size, nb_epoch=nb_epoch,
              shuffle=True, verbose=2, validation_data=(Xval, Yval),
              callbacks=callbacks, class_weight=class_weight)

    predictions_valid = model.predict(Xval.astype('float32'), batch_size=batch_size, verbose=2)
    score = log_loss(Yval, predictions_valid)
    print('Score log_loss: ', score)
    sum_score += score*len(mval)
    
    # Store valid predictions
    for i in range(len(mval)):
        yfull_train[mval[i]] = predictions_valid[i]

    models.append(model)



Start KFold number 1 from 20
('Split train: ', 4361, 4361)
('Split valid: ', 230, 230)
Train on 4361 samples, validate on 230 samples
Epoch 1/40
207s - loss: 2.3736 - acc: 0.4304 - val_loss: 1.7010 - val_acc: 0.5348
Epoch 2/40
221s - loss: 1.5178 - acc: 0.5648 - val_loss: 1.4823 - val_acc: 0.5696
Epoch 3/40
213s - loss: 1.2622 - acc: 0.6345 - val_loss: 1.4839 - val_acc: 0.6043
Epoch 4/40
209s - loss: 1.0250 - acc: 0.7012 - val_loss: 1.4607 - val_acc: 0.5826
Epoch 5/40
199s - loss: 0.8169 - acc: 0.7679 - val_loss: 1.5072 - val_acc: 0.5783
Epoch 6/40
196s - loss: 0.6751 - acc: 0.8104 - val_loss: 1.4502 - val_acc: 0.6174
Epoch 7/40
241s - loss: 0.5455 - acc: 0.8436 - val_loss: 1.4763 - val_acc: 0.6261
Epoch 8/40
217s - loss: 0.4407 - acc: 0.8734 - val_loss: 1.5042 - val_acc: 0.6522
Epoch 9/40
222s - loss: 0.3888 - acc: 0.8913 - val_loss: 1.7993 - val_acc: 0.6087
Epoch 10/40
182s - loss: 0.3469 - acc: 0.9007 - val_loss: 1.5697 - val_acc: 0.6261
Epoch 11/40
189s - loss: 0.3163 - acc: 0.9119

MemoryError: 
Apply node that caused the error: Dot22(Elemwise{Composite{Switch(i0, (i1 * i2), i3)}}[(0, 1)].0, dense_3_W.T)
Toposort index: 120
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(64L, 64L), (64L, 246016L)]
Inputs strides: [(256L, 4L), (4L, 256L)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{4}(Dot22.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
score = sum_score/len(Xtrall)
print("Log_loss train independent avg: ", score)

info_string = 'loss_' + str(score) + '_folds_' + str(10) + '_ep_' + str(nb_epoch)

In [ ]:
batch_size = 32
num_fold = 0
yfull_test = []
test_id = []
nfolds = len(models)

for i in range(nfolds):
    model = models[i]
    num_fold += 1
    print('Start KFold number {} from {}'.format(num_fold, nfolds))
    test_prediction = model.predict(X_test, batch_size=batch_size, verbose=2)
    yfull_test.append(test_prediction)
    


In [ ]:
a = np.array(yfull_test[0])
for i in range(1, nfolds):
    a += np.array(yfull_test[i])
a /= nfolds
test_res = a.tolist()

In [ ]:
info_string = 'loss_' + info_string \
                + '_folds_' + str(nfolds)

In [ ]:
params = {"0":"Automobile","1":"Comics_and_Cartoons","2":"Famous_Personalities","3":"Festivals_and_Occasions",
              "4":"Graffiti_and_Illustrations","5":"Movies_and_TV_shows","6":"Music","7":"Nature","8":"No_Theme",
              "9":"Patterns_and_Ethnic","10":"Signs_and_Symbols","11":"Spiritual","12":"Sports","13":"Superheroes",
              "14":"Typography","15":"Vintage"}
    
y = np.array([params[str(i)] for i in np.argmax(test_res, axis=1)])

In [ ]:
result1 = pd.DataFrame(X_test_id, columns=['id'])
result1.loc[:, 'Mobile_Theme'] = pd.Series(y, index=result1.index)
now = datetime.datetime.now()
sub_file = 'submission_keras_v4' +'_'+info_string+  '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
result1.to_csv(sub_file, index=False)